In [43]:
# Library
import pandas as pd
import numpy  as np
import  joblib

from sklearn.ensemble        import RandomForestClassifier
from sklearn.preprocessing   import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics         import accuracy_score, confusion_matrix

# Validação cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [44]:
heart_df = pd.read_csv('dataset/heart_tratado.csv', sep = ';', encoding = 'utf-8')
heart_df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,M,TA,110,264.0,0,Normal,132,N,1.2,Flat,1
913,68,M,ASY,144,193.0,1,Normal,141,N,3.4,Flat,1
914,57,M,ASY,130,131.0,0,Normal,115,Y,1.2,Flat,1
915,57,F,ATA,130,236.0,0,LVH,174,N,0.0,Flat,1


### Legenda

Age = Idade(anos)

Sex = sexo (0=M; 1=F)

Chest Pain Type = tipo de dor no peito (0=TA:angina típica; 1=ATA:angina atípica; 2=NAP:dor não anginosa; 3=ASY:assintomático)

Resting BP = pressão sanguínea em repouso (mmHg)

Cholesterol = colesterol sérico (mg/dl)

Fasting BS = açúcar no sangue (mg/dl) (0:Fasting BS < 120 mg/dl (não diabético); 1:Fasting BS > 120 mg/dl (diabético))

Resting ECG = eletrocardiograma em repouso (0=Normal; 1=ST:Anormalidade da onda ST-T; 2=LVH:hipertrofia ventricular esquerda)

Max HR = frequência cardíaca máxima

Exercise Angina = angina induzida por exercício (0=Não; 1=Sim)

Old Peak = depressão de ST induzida por exercício em relação ao repouso

ST_Slope = inclinação do segmento ST (0=UP; 1=Flap; 2=Down)

Heart Disease = Doença cardíaca (0=Não possui doença cardíaca; 1=Possui doença cardíaca)

### Previsores e alvo

In [45]:
previsores = heart_df.iloc[:, 0:11].values
alvo = heart_df.iloc[:, 11].values

# Transformando as variáveis categóricas nominais em variáveis categóricas ordinais
lista = [1,2,6,8,10]
for i in lista:
    previsores[:, i] = LabelEncoder().fit_transform(previsores[:, i])

# Dividindo dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(previsores, alvo, test_size = 0.3, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(641, 11)
(276, 11)
(641,)
(276,)


In [46]:
classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0, max_depth = 7, warm_start=True)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=500,
                       random_state=0, warm_start=True)

In [47]:
treino_cls = classifier.predict(X_train)

print(classifier)
print("\nTrain:")
print("Acurácia: %.2f%%" % (accuracy_score(y_train, treino_cls) * 100.0))
print(confusion_matrix(y_train, treino_cls))

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=500,
                       random_state=0, warm_start=True)

Train:
Acurácia: 95.48%
[[285  18]
 [ 11 327]]


In [48]:
teste_cls = classifier.predict(X_test)

print(classifier)
print("\nTest:")
print("Acurácia: %.2f%%" % (accuracy_score(y_test, teste_cls) * 100.0))
confusion_matrix(y_test, teste_cls)
print(confusion_matrix(y_test, teste_cls))

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=500,
                       random_state=0, warm_start=True)

Test:
Acurácia: 88.04%
[[ 93  14]
 [ 19 150]]


### Validação Cruzada

In [49]:
# Separando os dados em folds
kfold = KFold(n_splits = 5, shuffle = True, random_state = 5)

resultado = cross_val_score(classifier, previsores, alvo, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 87.24%


### Conlusão da aprendizagem

Classificador: RandomForestClassifier(criterion = 'entropy', max_depth = 7, n_estimators = 1000, random_state = 0, warm_start = True)
- Treino: Acurácia de 95.48% | Acertos: 612 Erros: 29
- Teste: Acurácia de 88.04% | Acertos: 243 Erros: 33
- Validação Cruzada: Acurácia Média de 87.24%

### Utilizando todos os dados para criar o classificador

In [50]:
classifier2 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0, max_depth = 7, warm_start=True)
classifier2.fit(previsores, alvo)

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=500,
                       random_state=0, warm_start=True)

In [51]:
cls2 = classifier2.predict(previsores)

print(classifier2)
print("Acurácia: %.2f%%" % (accuracy_score(alvo, cls2) * 100.0))
confusion_matrix(alvo, cls2)
print(confusion_matrix(alvo, cls2))

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=500,
                       random_state=0, warm_start=True)
Acurácia: 94.11%
[[377  33]
 [ 21 486]]


### Salvando dados para deploy

In [52]:
np.savetxt('dados/previsores.csv', previsores, delimiter = ',')
np.savetxt('dados/alvo.csv', alvo, delimiter = ',')
# Save the model as a pickle in a file
joblib.dump(classifier2, 'dados/classifier.pkl')

['dados/classifier.pkl']